<a href="https://colab.research.google.com/github/AmirKage/TDS-GroupProject24-25/blob/Group_work_AmirKage/Testing_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [ ]:
import json
with open(r"train2.json", "r") as read_file:
    train = json.load(read_file)

In [ ]:
train

[{'context': 'The Eiffel Tower is a wrought iron lattice tower located on the Champ de Mars in Paris, France. It was named after the engineer Gustave Eiffel, whose company designed and built the structure. The tower has become a global cultural icon of France and one of the most recognizable structures in the world. Construction of the tower began in January 1887, and it was completed on March 31, 1889. The tower was originally to be dismantled in 1909 after a 20-year concession agreement. However, its value as a radio broadcast transmitter saved it from demolition. The Eiffel Tower has three observation decks accessible to the public, including a restaurant. The top of the tower is 324 meters (1,063 ft) above ground.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Where is the Eiffel Tower located?',
    'answers': [{'text': 'Champ de Mars in Paris, France',
      'answer_start': 43}]},
   {'id': '00002',
    'is_impossible': False,
    'question': 'Who design

In [ ]:
# with open(r"test.json", "r") as read_file:
 #   test = json.load(read_file)


In [ ]:
# test

In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
#setting the type of models we will use and setting their names
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [ ]:
# setting model parameters
# we can set the parameters in the brackets such as "max_answer_length: int:50" or "n_best_size:int=20" this n parameter means that the model should give atleast 20 different answers to my question
# Configuring the model
#model_args = QuestionAnsweringArgs()
#model_args.train_batch_size = 16
#model_args.evaluate_during_training = True
#model_args.n_best_size=3  # this can be changed to get more number of answers
#model_args.num_train_epochs=5

In [ ]:
# this is required for visualization
!pip install wandb

In [ ]:
# we can also manually set the parameters
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [ ]:
# now we initialize the models, and train them

model = QuestionAnsweringModel(
    model_type="distilbert", model_name="distilbert-base-cased", args=train_args, use_cuda=False
)

# we used use_cuda=False to tell the model to use CPU instead of GPU

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
### Remove output folder
!rm -rf outputs

In [ ]:
# Train the model
model.train_model(train, eval_data=train)

# the output can be viewed through graphs by clicking the links for wandb, user might have to sign into wandb to view the data

convert squad examples to features:   0%|          | 0/12 [00:00<?, ?it/s]Could not find answer: 'tower located on the Champ de' vs. 'Champ de Mars in Paris, France'
Could not find answer: 'France. It was' vs. 'Gustave Eiffel'
Could not find answer: 'Renaissance artist Leonardo' vs. 'Leonardo da Vinci'
Could not find answer: 'about, the most parodied work' vs. 'the Louvre Museum in Paris'
Could not find answer: 'The Great Wall of China is a series of fortifications made of stone, brick, earth, wood, and other materials built along the historical' vs. 'a series of fortifications made of stone, brick, earth, wood, and other materials built along the historical northern borders of China'
Could not find answer: 'Several walls were being built as early' vs. 'during the Ming dynasty (1368–1644)'
Could not find answer: 'The Amazon rainforest is the world's largest' vs. 'the world's largest tropical rainforest'
Could not find answer: 'mi) of the' vs. 'Brazil'
add example index and unique id: 1

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/1 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 12/12 [00:00<00:00, 206.03it/s]

add example index and unique id: 100%|██████████| 12/12 [00:00<00:00, 37477.03it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(10,
 {'global_step': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'correct': [0, 1, 4, 4, 4, 4, 4, 4, 4, 4],
  'similar': [6, 8, 8, 8, 8, 8, 8, 8, 8, 8],
  'incorrect': [6, 3, 0, 0, 0, 0, 0, 0, 0, 0],
  'train_loss': [4.715978622436523,
   4.883520126342773,
   4.526403903961182,
   4.26401948928833,
   4.154877662658691,
   3.814406394958496,
   3.7175116539001465,
   3.4892334938049316,
   3.3281798362731934,
   3.2214841842651367],
  'eval_loss': [-0.0432809442281723,
   -0.2235424667596817,
   -0.3859598636627197,
   -0.5340449213981628,
   -0.6727779507637024,
   -0.7987247109413147,
   -0.9114620089530945,
   -1.0051244497299194,
   -1.0728322267532349,
   -1.1085997819900513]})

In [ ]:
# Evaluate the model
result, texts = model.eval_model(train)

/usr/local/lib/python3.11/dist-packages/simpletransformers/question_answering/question_answering_model.py:302: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  features = torch

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# to see the texts after evaluations
texts

{'correct_text': {'00003': '', '00006': '', '00009': '', '00012': ''},
 'similar_text': {'00001': {'truth': 'Champ de Mars in Paris, France',
   'predicted': '',
   'question': 'Where is the Eiffel Tower located?'},
  '00002': {'truth': 'Gustave Eiffel',
   'predicted': '',
   'question': 'Who designed and built the Eiffel Tower?'},
  '00004': {'truth': 'Leonardo da Vinci',
   'predicted': '',
   'question': 'Who painted the Mona Lisa?'},
  '00005': {'truth': 'the Louvre Museum in Paris',
   'predicted': '',
   'question': 'Where is the Mona Lisa currently displayed?'},
  '00007': {'truth': 'a series of fortifications made of stone, brick, earth, wood, and other materials built along the historical northern borders of China',
   'predicted': '',
   'question': 'What is the Great Wall of China?'},
  '00008': {'truth': 'during the Ming dynasty (1368–1644)',
   'predicted': '',
   'question': 'When was the majority of the existing wall constructed?'},
  '00010': {'truth': "the world's lar

In [ ]:
# to check results
result

{'correct': 4, 'similar': 8, 'incorrect': 0, 'eval_loss': -1.1085997819900513}

In [ ]:
# Make predictions with the model
to_predict = [
    {
        "context": "The Eiffel Tower is a wrought iron lattice tower located on the Champ de Mars in Paris, France. It was named after the engineer Gustave Eiffel, whose company designed and built the structure. The tower has become a global cultural icon of France and one of the most recognizable structures in the world. Construction of the tower began in January 1887, and it was completed on March 31, 1889. The tower was originally to be dismantled in 1909 after a 20-year concession agreement. However, its value as a radio broadcast transmitter saved it from demolition. The Eiffel Tower has three observation decks accessible to the public, including a restaurant. The top of the tower is 324 meters (1,063 ft) above ground.",
        "qas": [
            {
                "question": "What is the weight of the Eiffel Tower?'?",
                "id": "00003",
            }
        ],
    }
]

In [ ]:
answers, probabilities = model.predict(to_predict)



add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3682.44it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(answers)

[{'id': '00003', 'answer': ['empty']}]


In [ ]:
import json

# Load the questionnaire file
with open('questionnaire5.json', 'r') as file:
    questionnaire = json.load(file)

# Print questions to understand the structure
for item in questionnaire:
    print(f"Question: {item['question']}")


Question: Customer type
Question: Customer satisfaction
Question: Size of the trade fair team (on average)
Question: CRM-System
Question: Productinterests
Question: Searches a solution for


RESTART FROM HERE!!!1

In [2]:
from transformers import AutoTokenizer
import pandas as pd

# Example responses for Questions 1, 2, 3, 4, 5, and 6

questions = {
    "Q1":"What is your preferred CRM platform?",
    "Q2":"Which product feature do you use most frequently?",
    "Q3":"What challenges do you face while organizing trade fair teams?",
    "Q4":"What improvements would you suggest for CRM systems?",
    "Q5":"How does your company currently manage and improve CRM data quality?",
    "Q6":"If you could design a custom CRM feature, what would it be, and why?"
}

responses = {
    "Q1": [
    "I mostly use Salesforce, especially for analytics and scalability.",
    "HubSpot is great because it's user-friendly and cost-effective.",
    "We rely on Microsoft Dynamics for its integration with Office tools.",
    "Pipedrive works perfectly for our small sales team.",
    "I prefer Close.io due to its simplicity and focus on sales.",
    "Salesforce has been our go-to CRM for years—it’s reliable.",
    "HubSpot meets our marketing needs and is easy to onboard new employees.",
    "Microsoft Dynamics aligns well with our ERP system.",
    "SAP Sales Cloud is our choice for its enterprise-level features.",
    "Pipedrive is ideal for startups like ours.",
    "I use Salesforce, although it can be a bit complex at times.",
    "HubSpot offers the best balance of features and affordability for us.",
    "Close.io simplifies our process of tracking and closing leads.",
    "We’ve adopted Microsoft Dynamics for its strong reporting tools.",
    "Salesforce is robust but sometimes feels overwhelming.",
    "HubSpot makes it easy to align marketing and sales teams.",
    "Pipedrive’s visual pipeline is a game-changer for our workflow.",
    "SAP Sales Cloud provides the scalability we need for our growing team.",
    "Microsoft Dynamics integrates seamlessly with our existing tools.",
    "Salesforce offers unmatched customization, though it requires expertise.",
    "HubSpot’s support team is excellent, and the product keeps improving.",
    "Pipedrive is affordable and gets the job done without too many bells and whistles.",
    "Close.io fits our needs as a fast-paced, sales-driven organization.",
    "We stick with Salesforce because of its third-party app integrations.",
    "Microsoft Dynamics is essential for our large organization’s structure.",
    "SAP Sales Cloud has robust AI-driven recommendations we find helpful.",
    "HubSpot allows us to track customer interactions in one place effectively.",
    "Close.io’s focus on sales pipelines has been very effective for our team.",
    "Pipedrive’s customization options make it the best fit for us.",
    "Salesforce remains the industry standard, so it’s hard to switch away."
    ],
    "Q2": [
    "The lead management feature is my go-to.",
    "I rely heavily on the task automation tools.",
    "Email tracking is the most useful for me.",
    "I always use the contact management feature.",
    "The analytics dashboard is where I spend most of my time.",
    "Data enrichment tools are what we use the most.",
    "The pipeline visualization feature is my favorite.",
    "I frequently use the meeting scheduling tool.",
    "The integration with marketing automation tools is invaluable.",
    "Custom reporting is the feature I use every day.",
    "For me, it’s all about the customer segmentation tools.",
    "I depend on the document storage feature.",
    "The mobile app sync is my most-used feature.",
    "Lead scoring tools are the ones I interact with the most.",
    "I use the sales forecasting feature weekly.",
    "Customizable workflows are the most helpful.",
    "Our team heavily uses the team collaboration tools.",
    "The real-time notifications are a lifesaver.",
    "I couldn’t work without the multi-channel support feature.",
    "The activity tracking tools are the most essential for me.",
    "I use the CRM’s chat integration feature regularly.",
    "The API access for integrating with our other tools is vital.",
    "Email campaign management is my daily focus.",
    "I use the file sharing feature almost every day.",
    "The bulk data upload tool is our most-used feature.",
    "Our marketing team loves the A/B testing tools.",
    "The feature I rely on the most is team performance tracking.",
    "I frequently use the call logging and recording tools.",
    "The audit trail feature is my go-to for compliance.",
    "Our team depends heavily on the shared calendar functionality."
    ],
    "Q3": [
    "Coordinating schedules among team members is always tough.",
    "Finding a balance between team size and efficiency is challenging.",
    "It’s difficult to ensure everyone has the right materials.",
    "Keeping communication clear during the event is a big hurdle.",
    "Managing travel and accommodation logistics is stressful.",
    "Allocating tasks fairly is often a source of friction.",
    "Training new members before trade fairs is time-consuming.",
    "Tracking team performance during the event is a challenge.",
    "Handling last-minute changes to the schedule can be chaotic.",
    "Ensuring that everyone adheres to the budget is a major issue.",
    "It’s hard to track the leads collected by each team member.",
    "Finding enough experienced staff for the event is a recurring problem.",
    "Aligning objectives across departments is challenging.",
    "Dealing with unexpected technical issues is always tough.",
    "Creating a follow-up process for leads post-event is complicated.",
    "Coordinating shifts for the booth team is exhausting.",
    "Balancing in-person and virtual participation is tricky.",
    "Ensuring we have enough promotional materials is difficult.",
    "Training the team on the trade fair tools is a challenge.",
    "Getting everyone to adhere to the event’s compliance requirements is hard.",
    "Making sure the team understands our product is an ongoing task.",
    "It’s a struggle to measure the ROI of our participation.",
    "Avoiding burnout for team members during the event is critical.",
    "Managing cultural differences in international fairs can be tricky.",
    "Tracking who has interacted with which leads is a nightmare.",
    "Handling language barriers at global events is tough.",
    "It’s hard to make sure our booth setup aligns with the branding.",
    "Motivating the team to perform their best is a challenge.",
    "Ensuring that everyone has access to event updates is hard.",
    "Following up on potential client feedback during the fair is difficult."
    ],
    "Q4": [
    "Make the user interface more intuitive and beginner-friendly.",
    "Add better AI-driven insights for sales forecasting.",
    "Include more robust data visualization tools.",
    "Improve integration with third-party tools like Slack or Trello.",
    "Add offline access with seamless syncing once back online.",
    "Focus on faster loading times for large datasets.",
    "Provide more detailed training materials and tutorials.",
    "Simplify the process for creating custom reports.",
    "Add multi-language support for global teams.",
    "Improve the mobile app’s functionality and reliability.",
    "Introduce a built-in feature for lead prioritization.",
    "Enhance security features for sensitive customer data.",
    "Allow for easier customization of dashboards.",
    "Include better tools for automating follow-up emails.",
    "Add features to manage recurring client interactions.",
    "Offer more flexible API access for custom integrations.",
    "Improve the scalability for growing teams.",
    "Provide real-time collaboration tools for team members.",
    "Ensure better cross-platform compatibility.",
    "Integrate video conferencing directly into the CRM.",
    "Enable a drag-and-drop interface for organizing pipelines.",
    "Offer a predictive text feature for faster data entry.",
    "Add a more robust search feature for finding specific records.",
    "Allow automatic syncing with social media platforms.",
    "Make data cleansing easier with automated suggestions.",
    "Improve task management tools for better tracking.",
    "Add a feature for tracking competitors’ activities.",
    "Include a voice assistant for hands-free operation.",
    "Offer gamification features to motivate sales teams.",
    "Reduce subscription costs or offer more flexible pricing tiers."
    ],
    "Q5": [
    "We conduct quarterly data audits to remove outdated records.",
    "Our team regularly reviews and cleanses the database.",
    "We use automated tools to identify and merge duplicate entries.",
    "Our CRM is synced with external systems to maintain consistency.",
    "Data entry is restricted to trained staff to ensure accuracy.",
    "We’ve implemented mandatory data validation fields in forms.",
    "We hire consultants to perform regular data quality checks.",
    "Our company uses AI tools for real-time data enrichment.",
    "We encourage employees to flag and report incorrect data.",
    "Automated workflows ensure that incomplete data is flagged.",
    "Our CRM automatically highlights inconsistencies for review.",
    "We hold monthly training sessions on best practices for data entry.",
    "Our CRM has built-in rules to prevent incorrect data input.",
    "We have a dedicated data quality team to oversee the process.",
    "All new data entries are cross-referenced with existing records.",
    "We partner with a data cleansing service for large-scale updates.",
    "Data backups are maintained weekly to prevent loss.",
    "Our company uses API integrations to pull verified data sources.",
    "Regular performance reviews include data quality as a metric.",
    "We have set up alerts for key data changes to ensure monitoring.",
    "All leads are validated before they’re added to the CRM.",
    "Our CRM uses machine learning to suggest corrections.",
    "We periodically request clients to update their own information.",
    "The sales team collaborates with IT to maintain database integrity.",
    "We use conditional logic in forms to reduce human error.",
    "Data is categorized clearly to avoid duplication.",
    "We review historical data annually to remove irrelevant information.",
    "A centralized team is responsible for finalizing all key data updates.",
    "Our CRM tracks changes so we can identify problematic patterns.",
    "We outsource complex data cleansing tasks to specialized vendors."
    ],
    "Q6": [
    "A predictive analytics tool to identify which leads are most likely to convert.",
    "A built-in feature for tracking and improving team collaboration during sales cycles.",
    "A voice-to-text integration for logging notes after client meetings.",
    "An AI assistant that suggests follow-up actions based on customer behavior.",
    "A real-time dashboard showing the health of our sales pipeline.",
    "A tool that automatically cleans up and enriches data on a weekly basis.",
    "A feature that analyzes customer feedback and creates actionable insights.",
    "Integration with accounting tools to track customer invoices and payments.",
    "A seamless integration with social media to monitor customer interactions.",
    "A module for competitor analysis based on market trends and customer interactions.",
    "An offline mode with auto-sync for field teams who don’t always have internet access.",
    "A timeline feature that visually shows the customer journey end-to-end.",
    "An AI-powered chatbot that helps prospects navigate product features in real time.",
    "A gamification system to reward team members for achieving milestones.",
    "A CRM-wide search bar to quickly find any record or piece of information.",
    "A calendar integration that auto-schedules meetings based on mutual availability.",
    "A feature that generates automatic summaries of customer emails.",
    "A more advanced segmentation tool for highly specific targeting.",
    "A drag-and-drop workflow builder for creating custom sales processes.",
    "A system that recommends upsell and cross-sell opportunities based on data trends.",
    "An integrated feedback loop for customers to submit their suggestions directly.",
    "A real-time notification system for significant customer activities.",
    "A feature to generate customer health scores based on engagement data.",
    "A built-in survey tool that logs responses directly into the CRM.",
    "An intuitive tool for tracking multi-team interactions with the same client.",
    "A dashboard that predicts potential churn rates and recommends retention strategies.",
    "A CRM-integrated document editor for collaboration on proposals and contracts.",
    "A feature that tracks competitor pricing and adjusts our proposals accordingly.",
    "An AI tool that recommends optimal meeting times for large teams.",
    "A seamless voice recognition feature for hands-free CRM navigation."
    ]
}

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Tokenize all responses
tokenized_data = {}
for question, answers in responses.items():
    tokenized = tokenizer(
        answers,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    tokenized_data[question] = {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"]
    }

print("Tokenization complete. Data is ready for training.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Tokenization complete. Data is ready for training.


In [3]:
# Example categorical labels for Question 1
q1_labels = ["Salesforce", "HubSpot", "Pipedrive", "Microsoft Dynamics", "Close.io", "SAP Sales Cloud"]
q2_labels = [
    "Lead management", "Task automation", "Email tracking", "Contact management",
    "Analytics dashboard", "Pipeline visualization", "Meeting scheduling", "Custom reporting"]

q3_labels = [
    "Schedule Coordination",
    "Team Efficiency Balance",
    "Schedule Changes",
    "Budget Adherence",
    "Promotional Material Management",
    "Compliance Adherence",
    "ROI Measurement",
    "Language Barriers"
]
q4_labels = ["User Interface & Usability",
    "AI & Data Insights",
    "Third-party Integration",
    "Mobile & Offline Functionality",
    "Customization & Personalization",
    "Collaboration & Scalability"]

q5_labels = [  "Data Audits & Cleansing",
    "Automation & AI Integration",
    "Data Validation & Accuracy",
    "Data Consistency & Syncing",
    "Training & Best Practices",
    "Collaboration & Monitoring"]

q6_labels = [ "AI & Predictive Tools",
    "Team Collaboration & Communication",
    "Automation & Data Enrichment",
    "Customer Insights & Feedback",
    "Customization & Flexibility",
    "Cross-Platform Integration"]

# Add labels for other single-select questions as needed
labels = {
    "Q1": q1_labels,
    "Q2": q2_labels,
    "Q3": q3_labels,
    "Q4": q4_labels,
    "Q5": q5_labels,
    "Q6": q6_labels
    # Add labels for Q2, Q3, etc., if applicable
}

# Map responses to numerical labels for Q1
# q1_labels = []
# for response in q1_responses:
 #   for idx, option in enumerate(q1_options):
  #      if option.lower() in response.lower():
   #         q1_labels.append(idx)
    #        break

# Map responses to numerical labels for Q2
# q2_labels = []
 #for response in q2_responses:
  #  for idx, option in enumerate(q2_options):
   #     if option.lower() in response.lower():
    #        q2_labels.append(idx)
     #       break

# Print the labels for Q1 and Q2
print("Q1 Labels:", q1_labels)  # Output: Numerical labels for Q1
print("Q2 Labels:", q2_labels)  # Output: Numerical labels for Q2


Q1 Labels: ['Salesforce', 'HubSpot', 'Pipedrive', 'Microsoft Dynamics', 'Close.io', 'SAP Sales Cloud']
Q2 Labels: ['Lead management', 'Task automation', 'Email tracking', 'Contact management', 'Analytics dashboard', 'Pipeline visualization', 'Meeting scheduling', 'Custom reporting']


In [7]:
# Store all responses in a dictionary, using responses from your existing 'responses' dictionary
responses_data = {
    "Q1": {"responses": responses["Q1"], "labels": q1_labels},  # Use responses["Q1"] instead of q1_responses
    "Q2": {"responses": responses["Q2"], "labels": q2_labels},  # Use responses["Q2"] instead of q2_responses
    "Q3": {"responses": responses["Q3"], "labels": q3_labels},  # Use responses["Q3"] instead of q3_responses
    "Q4": {"responses": responses["Q4"], "labels": q4_labels},  # Use responses["Q4"] instead of q4_responses
    "Q5": {"responses": responses["Q5"], "labels": q5_labels},  # Use responses["Q5"] instead of q5_responses
    "Q6": {"responses": responses["Q6"], "labels": q6_labels}  # Use responses["Q6"] instead of q6_responses
    # Add other questions and their responses here
}

# Print the final dataset structure
for question, data in responses_data.items():
    print(f"{question}:")
    print("Responses:", data["responses"])
    print("Labels:", data["labels"])

Q1:
Responses: ['I mostly use Salesforce, especially for analytics and scalability.', "HubSpot is great because it's user-friendly and cost-effective.", 'We rely on Microsoft Dynamics for its integration with Office tools.', 'Pipedrive works perfectly for our small sales team.', 'I prefer Close.io due to its simplicity and focus on sales.', 'Salesforce has been our go-to CRM for years—it’s reliable.', 'HubSpot meets our marketing needs and is easy to onboard new employees.', 'Microsoft Dynamics aligns well with our ERP system.', 'SAP Sales Cloud is our choice for its enterprise-level features.', 'Pipedrive is ideal for startups like ours.', 'I use Salesforce, although it can be a bit complex at times.', 'HubSpot offers the best balance of features and affordability for us.', 'Close.io simplifies our process of tracking and closing leads.', 'We’ve adopted Microsoft Dynamics for its strong reporting tools.', 'Salesforce is robust but sometimes feels overwhelming.', 'HubSpot makes it easy

In [8]:
import torch
from torch.utils.data import Dataset

class CRMResponsesDataset(Dataset):
    def __init__(self, tokenized_data, labels=None):
        self.input_ids = tokenized_data["input_ids"]
        self.attention_mask = tokenized_data["attention_mask"]

        # Ensure labels list has the same length as input data
        # If labels are provided, use them. Otherwise, create a list of None values with the correct length.
        self.labels = labels if labels else [None] * len(self.input_ids)

        # Adjust labels length to match input_ids length if necessary
        if len(self.labels) != len(self.input_ids):
            # Extend the labels list with None values if it's shorter
            if len(self.labels) < len(self.input_ids):
                self.labels.extend([None] * (len(self.input_ids) - len(self.labels)))
            # Truncate the labels list if it's longer
            else:
                self.labels = self.labels[:len(self.input_ids)]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        item = {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
        }
        if self.labels[idx] is not None:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

# Example dataset for Question 1
q1_dataset = CRMResponsesDataset(
    tokenized_data["Q1"],
    labels=[next((i for i, label in enumerate(q1_labels) if label.lower() in response.lower()), -1)
            for response in responses["Q1"]]
)

q2_dataset = CRMResponsesDataset(
    tokenized_data["Q2"],
    labels=[next((i for i, label in enumerate(q2_labels) if label.lower() in response.lower()), -1)
            for response in responses["Q2"]]
)

In [9]:
# Step 4: Training a RoBERTa Model

import torch
from torch.utils.data import random_split
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

# Assuming q1_dataset is your complete dataset for Question 1
total_samples = len(q1_dataset)
train_size = int(0.75 * total_samples)
eval_size = total_samples - train_size

# Split the dataset
train_dataset, eval_dataset = random_split(q1_dataset, [train_size, eval_size])



In [10]:
# --- Training ---

# Load pre-trained RoBERTa model for training
training_model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(q1_labels))

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Create a Trainer instance for training
trainer = Trainer(
    model=training_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train the model
trainer.train()



model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,1.836592
2,No log,1.827503
3,No log,1.817530
4,No log,1.811435
5,No log,1.808407


TrainOutput(global_step=10, training_loss=1.808656883239746, metrics={'train_runtime': 83.8385, 'train_samples_per_second': 1.312, 'train_steps_per_second': 0.119, 'total_flos': 1187125715160.0, 'train_loss': 1.808656883239746, 'epoch': 5.0})

In [11]:
# --- Evaluation with a Different Model ---

# Load a different pre-trained RoBERTa model for evaluation (e.g., roberta-large)
eval_model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=len(q1_labels))

# Create a Trainer instance for evaluation
# We are reusing training_args here, but you can create a separate TrainingArguments for evaluation if needed
eval_trainer = Trainer(
    model=eval_model,
    args=training_args,
    eval_dataset=eval_dataset
)

# Evaluate the model
eval_results = eval_trainer.evaluate()

print("Evaluation Results:", eval_results)


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation Results: {'eval_loss': 1.93157160282135, 'eval_model_preparation_time': 0.0095, 'eval_runtime': 2.9714, 'eval_samples_per_second': 2.692, 'eval_steps_per_second': 0.337}


In [17]:
# UNSURE ABOUT THIS BUT I'M TRIED TO ASK A QUESTION TO CHECK PERFORMANCE

question_text = "Do you use CRM?" # User's question

# Tokenize the question text
inputs = tokenizer(question_text, return_tensors="pt")

# Make prediction using the trained model
# Assuming 'eval_model' is your trained model
with torch.no_grad():
    outputs = eval_model(**inputs)

# Get the predicted category
predicted_category_index = torch.argmax(outputs.logits).item()
predicted_category = q1_labels[predicted_category_index]  # Get category label from q1_labels

print(f"Question: {question_text}")
print(f"Predicted Category: {predicted_category}")

Question: Do you use CRM?
Predicted Category: SAP Sales Cloud
